In [1]:
import numpy as np
from preprocessing.text import tokenize
from models.llm import load_llm
from models.classifier import load_classifier
from tqdm import tqdm
from preprocessing.image import preprocess_and_check_image

/home/nightstalker/miniconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from database.connect import conn

In [3]:
llm = load_llm()
trainer, tokenizer = load_classifier()

In [4]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM Baby_Bath_Skin_and_Grooming") 
rows = cursor.fetchall()

In [5]:
IMAGE_INDEX = 4
PRIMARY_KEY = ['id']

columns = [col[0] for col in cursor.description]
pk_index = [i for i in range(len(columns)) if columns[i] in PRIMARY_KEY]

columns, pk_index

(['id',
  'name',
  'main_category',
  'sub_category',
  'image',
  'ratings',
  'no_of_ratings',
  'discount_price',
  'actual_price'],
 [0])

In [6]:
results_text = []
PK = []
indices = []

for i in tqdm(range(len(rows[:10]))):    
    image = preprocess_and_check_image(rows[i][IMAGE_INDEX])
    if image:
        prompt = "Analyze the picture, and the row data.\nDoes it have a baby picture on it that is seen easily in the package?"
        text_data = " | ".join([f"{columns[j]}: {rows[i][j]}" for j in range(len(columns)) if j != IMAGE_INDEX])
        result = llm.generate_content(
            [image, "\n\n", f"Row data: {text_data}\n\n", prompt]
        )
        results_text.append(result.text)
        dataset_subset = tokenize(result.text, tokenizer)
        predictions = trainer.predict(dataset_subset)

        predicted_class = np.argmax(predictions.predictions, axis=1)[0]
        
        if predicted_class <= 2:
            indices.append(i)
            PK.append([rows[i][j] for j in pk_index])

  0%|                                                                                                                                                                                                                  | 0/10 [00:00<?, ?it/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mehdinejjar (mehdinejjar-al-akhawayn-university). Use `wandb login --relogin` to force relogin


 10%|████████████████████▏                                                                                                                                                                                     | 1/10 [00:04<00:43,  4.85s/it]

 20%|████████████████████████████████████████▍                                                                                                                                                                 | 2/10 [00:07<00:30,  3.76s/it]

 30%|████████████████████████████████████████████████████████████▌                                                                                                                                             | 3/10 [00:10<00:21,  3.12s/it]

...

 60%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                                | 6/10 [00:17<00:10,  2.60s/it]

 70%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                            | 7/10 [00:20<00:07,  2.64s/it]

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                        | 8/10 [00:25<00:07,  3.51s/it]

 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                    | 9/10 [00:28<00:03,  3.21s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:30<00:00,  3.02s/it]


In [7]:
results_text, PK, indices

(["Yes, there is a clear picture of a baby on the LuvLap Liquid Cleanser bottle, prominently displayed on the front label.  The baby is shown playing with a toy, likely to reinforce the product's association with baby items.\n",
  'No, the product does not feature a picture of a real baby. It has cartoon illustrations of sea creatures like a narwhal, shark, octopus, fish, and turtle.  The cushion insert does have a simple narwhal design on it, but this is also a cartoon and not a photo.\n',
  "Yes, the packaging clearly features a simple illustration of a baby's head. It's located near the top of the box and on the bottle, making it easily visible.",
  'No, there is no baby picture on the packaging.  There is a picture of lotus and meadowsweet flowers.',
  "No, there isn't a clear, easily seen picture of a baby on the Cetaphil Baby Mild Bar packaging. There's a stylized, almost abstract, illustration of a baby being bathed/washed, but no actual photographic image of a baby.  The illust

In [8]:
rows_filtered = [rows[i] for i in indices]

In [9]:
rows_filtered

[(11,
  'Luv Lap Liquid Cleanser, Anti-Bacterial, Food Grade, For Baby Bottles, Accessories and Vegetables, 1.5ltr',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/71p5fCtiCnL._AC_UL320_.jpg',
  Decimal('4.40'),
  '5,883',
  '₹509',
  '₹669'),
 (12,
  'Luv Lap LuvLap Aqua Tales Baby Bather for Baby 0-12 Months, New Born Baby Bath Chair, 3 Position Adjustable, Washable Soft...',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/91JNkTWv4TL._AC_UL320_.jpg',
  Decimal('4.10'),
  '3,200',
  '₹899',
  '₹1,599'),
 (13,
  'The Moms Co. Natural Baby Massage Oil with 10 Oils - Sesame Oil, Avocado, Organic Almond, Organic Jojoba, Organic Chamomil...',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/71EQE76D+hL._AC_UL320_.jpg',
  Decimal('4.40'),
  '2,265',
  '₹331',
  '₹474'),
 (14,
  'Chicco Baby Moments Mild Body Wash Refresh, New Advanced Formula Wi

In [10]:
PK

[[11], [12], [13], [14], [15], [16], [18], [19], [20], [21]]